ポートフォリオ配分、目標値をGodelyの例の式の形で計算する。パラメータに対して進化的なアルゴリズムを使う。

各資産の評価を行い、売却量・売却価格・買取量・買取価格のオファーを出す

オファー同士でマッチング


ポートフォリオ配分戦略には、模倣の要素を入れる。

家計の労働所得を対数正規分布になるように分布させ、純資産が一定値を超えたら金融市場に参加、純資産が一定値を下回ったら保有する預金以外の金融資産を売って金融市場から撤退

In [1]:
using StatsPlots

Julia本を手元に

型付けは練習のために過剰なくらいで。

多重ディスパッチ、ブロードキャスティング、で読みやすさと行動方程式の書き換えやすさを両立したい

In [2]:
abstract type Agent end

mutable struct HouseHold <: Agent
    M::Float64      #   預金
    L::Float64      #   借入
    peE::Vector{Float64}    #   株式保有総額
    E::Vector{Float64}      #   株式保有量
    GB::Float64             #   国債
    CB::Vector{Float64}     #   社債
    λ_arr::Matrix{Float64}  #   ポートフォリオ目標を決めるための式のパラメータ
end
mutable struct Bank <: Agent
    H::Float64      #   HPM
    M::Float64      #   預金
    L::Float64      #   借入
    peE::Vector{Float64}    #   株式保有総額
    E::Vector{Float64}      #   株式保有量
    GB::Float64             #   国債
    CB::Vector{Float64}     #   社債
    λ_arr::Matrix{Float64}  #   ポートフォリオ目標を決めるための式のパラメータ
end
mutable struct Firm <: Agent
    M::Float64      #   預金
    L::Float64      #   借入
    E::Float64      #   株式発行量
    SC::Float64     #   資本金
    CB::Float64     #   社債
end
mutable struct IntegratedGovernment <: Agent
    H::Float64      #   HPM
    GB::Float64     #   国債
end


In [ ]:
function change_λ_set(λ_arr)   #   オレンジのポートフォリオ方程式から、単項式で書かれている要素を外生的にランダムに変化させ、多項式で書かれている要素をその値を使って計算しなおす
    if rand() < 0.1
        I, J = size(λ_arr)
        pi, pj, Δλ = rand(1:I), rand(1:J), 0.01*randn()
        while true
            if pi == 1 and pj == 1
                pi, pj = rand(1:I), rand(1:J)
            elseif pi + 1 == pj
                pi, pj = rand(1:I), rand(1:J)
            elseif pi == 1 and pj == J
                pi, pj = rand(1:I), rand(1:J)
            else
                break
            end
        end
        if pj == 1
            if 0 <= λ_arr[pi,pj] + Δλ <= 1 && 0 <= λ_arr[1,pj] - Δλ <= 1
                λ_arr[pi,pj] += Δλ
                λ_arr[1,pj] = 1 - sum(λ_arr[2:end,pj])
            end
        elseif pj == L
            λ_arr[pi,pj] += Δλ
            λ_arr[1,pj] = - sum(λ_arr[2:end,pj])
        else
            if λ_arr[pi,pj] + Δλ <= 0
                λ_arr[pi,pj] += Δλ
                λ_arr[pj,pi] += Δλ
                λ_arr[pi,pi] = -sum(λ_arr[1:end,pi]) + λ_arr[pi,pi]
                λ_arr[pj,pj] = -sum(λ_arr[1:end,pj]) + λ_arr[pj,pj]
            end
        end
    end
end

In [ ]:
function generate_init_λ_arr_lst(Nh)
    init_λ_arr_lst = [zeros(4,6) for _ = 1:Nh]
    for λ_arr in init_λ_arr_lst
        λ_arr[1,1] = 1.0
    end
    for t = 1:100
        for λ_arr in init_λ_arr_lst
            change_λ_set(λ_arr)
        end
    end
    return init_λ_arr_lst  
end

In [ ]:
#   各種エージェントの初期値を定義
Nh, Nb, Nf, Nig = 1000, 5, 10, 1

#   家計群の初期値
init_λ_arr_lst = generate_init_λ_arr_lst(Nh)
households = [HouseHold(
    100.0, 0.0, zeros(Nf), zeros(Nf), 1.0, zeros(Nf), init_λ_arr_lst[i],
)for i = 1:Nh]

In [6]:
#   金融資本収益率が高いλセットを高確率で、金融資本収益率が低いλセットを低確率で、マネする関数
function imitate_λ_set(households::HouseHold)

Matrix{Float64} (alias for Array{Float64, 2})

In [11]:
s = Set{Symbol}([:x, :y, :z])

Set{Symbol} with 3 elements:
  :y
  :z
  :x

In [14]:
dic = Dict{Symbol, Float64}(
    :a => 1.0,
    :b => 2.0,
    :c => 3.0,
    :d => 4.0,
)

Dict{Symbol, Float64} with 3 entries:
  :a => 1.0
  :b => 2.0
  :c => 3.0

In [25]:
ieqs = Set{Expr}([
    :(x - a*y),
    :(z - (b*x + c*y)),
    :(z - d),
])

Set{Expr} with 3 elements:
  :(z - (b * x + c * y) = begin…
  :(z - d = begin…
  :(x - a * y = begin…